# TMDB

https://api.themoviedb.org/3/movie/top_rated?api_key=9047074b33a9c918ef32418caeaeeb07&language=en-US&page=1

##### Pulling data without genres

In [36]:
import pandas as pd
import requests

df = pd.DataFrame()

# Looping through 500 pages. Each page has 20 movies.
for i in range(1,501):
    response = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key=9047074b33a9c918ef32418caeaeeb07&language=en-US&page={}'.format(i))
    temp_df = pd.DataFrame(response.json()['results'])[['id','title','overview','release_date','popularity','vote_average','vote_count']]
    df = df.append(temp_df,ignore_index=True)

In [55]:
print(df.shape)
df.head(3)

(10000, 7)


,id,title,overview,release_date,popularity,vote_average,vote_count
0,238,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,125.665,8.7,18577
1,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,105.936,8.7,24532
2,240,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,58.687,8.6,11218


##### Pulling data with genres

In [44]:
import tmdbsimple as tmdb

In [45]:
tmdb.API_KEY = '9047074b33a9c918ef32418caeaeeb07'

In [47]:
search = tmdb.Search()

In [48]:
genre = tmdb.Genres()

In [49]:
genres_lst = genre.movie_list()

In [50]:
search.movie(query='The Avengers')['results'][0]['genre_ids']

[878, 28, 12]

In [53]:
print(genres_lst)

{'genres': [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}, {'id': 99, 'name': 'Documentary'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 14, 'name': 'Fantasy'}, {'id': 36, 'name': 'History'}, {'id': 27, 'name': 'Horror'}, {'id': 10402, 'name': 'Music'}, {'id': 9648, 'name': 'Mystery'}, {'id': 10749, 'name': 'Romance'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 10770, 'name': 'TV Movie'}, {'id': 53, 'name': 'Thriller'}, {'id': 10752, 'name': 'War'}, {'id': 37, 'name': 'Western'}]}


In [54]:
def genre_labeller(movie_title):

    """
    This function takes a movie title as a parameter
    and returns a list of genres associated with respected movie

    """
    
    lst = []
    
    #the api could have multiple results
    #this function tries different results if genre list is empty in first result
    #up to three results
    try:
        for i in search.movie(query=movie_title)['results'][0]['genre_ids']: #loops through list of genre ids
            for x in genres_lst['genres']: #loops through list of genres
                if x['id'] == i: #appends genre to lst if id matches the id from the query
                    lst.append(x['name'])
                else:
                    continue
    except:
        pass

    if len(lst) == 0: #tries another query if the first one fails, up to three times
        try:
            for i in search.movie(query=movie_title)['results'][1]['genre_ids']:
                for x in genres_lst['genres']:
                    if x['id'] == i:
                        lst.append(x['name'])
                    else:
                        continue
        except:
            pass

    if len(lst) == 0:
        try:
            for i in search.movie(query=movie_title)['results'][2]['genre_ids']:
                for x in genres_lst['genres']:
                    if x['id'] == i:
                        lst.append(x['name'])
                    else:
                        continue
        except:
            pass

    #some movie titles have 'the' before title name
    if len(lst) == 0:
        try:
            for i in search.movie(query='The '+ movie_title)['results'][0]['genre_ids']:
                for x in genres_lst['genres']:
                    if x['id'] == i:
                        lst.append(x['name'])
                    else:
                        continue
        except:
            pass

    
    return lst

In [56]:
df['genre'] = df['title'].apply(lambda x: genre_labeller(x))

In [59]:
df.loc[:, ['title', 'overview', 'genre']]

,title,overview,genre
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[Drama, Crime]"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[Drama, Crime]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[Drama, Crime]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[Drama, History, War]"
4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[Comedy, Drama, Romance]"
...,...,...,...
9995,Urban Legend,A college campus is plagued by a vicious seria...,"[Horror, Thriller]"
9996,The Order,"For centuries, a secret Order of priests has e...","[Drama, Fantasy, Horror, Mystery, Romance, Thr..."
9997,247 Degrees Fahrenheit,Four friends travel to a lakeside cabin for a ...,"[Horror, Thriller]"
9998,The Stepford Wives,"What does it take to become a Stepford wife, a...","[Horror, Mystery, Science Fiction, Thriller]"


In [60]:
df.to_csv("C:\\Users\\shubh\\Desktop\\Important_IPYNB_files\\NLP\\Other_files\\tmdb_data.csv")